# <center>Deep learning with PyTorch</center>

- Dataset
- Dataloader
- Define Network
- Train/test loop
- Loss function and optimizer
- Train the network

## 1. Install packages into environment

Installation instructions for PyTorch can be found at https://pytorch.org/

Example:
> `$ conda create --name ml_tut python=3.10` <br>
> `$ conda activate ml_tut`<br>
> `$ conda install pytorch torchvision torchaudio cpuonly -c pytorch`<br>

Other packages used in this tutorial are numpy, matplotlib, scikit-learn, and torchviz (requires graphviz) <br>

On Linux you can install graphviz with Anaconda and torchviz with pip.
> `$ conda install -c conda-forge pygraphviz`<br>
> `$ pip install torchviz`<br>

**Note**: The installation process may be different for Windows users. And this notebook has only been tested on Linux (Ubuntu) with Python 3.9.

## 2. Download the dataset

There are many built-in image/video datasets available in PyTorch. They also have some for tasks other than classification such as object dectection, segmentation, etc. <br><br>
Full list of what is available can be found at https://pytorch.org/vision/main/datasets.html.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

training_data

In [ ]:
training_data.classes

In [ ]:
img, im_class = training_data[0]

npimg = img.numpy()
plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
plt.show()

In [ ]:
training_data.classes[im_class]

We will also need a dataloaders to load the train/test data from disk into memory.

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## 3. Define the network and train/test loops

We are just going to use a simple feed forward network with 3 layers.

In [ ]:
training_data[0][0].shape   # first dim is 1 b/c these are grayscale imgs

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)  # could have used torch.flatten(x,1)
        logits = self.linear_relu_stack(x)
        return logits
    

#### Key components that we need

- Model to train
- Loss function
- An optimizer
- Training loop to fit model parameters
- Testing loop to evaluate how well model fits

In [ ]:
# will want to move data to GPU later (if available)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
learning_rate = 1e-3    # step size for gradient descent

model = NeuralNetwork().to(device)  # move model to GPU if present

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 4. Training the network

In [ ]:
# will want to move data to GPU later (if available)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
learning_rate = 1e-3    # step size for gradient descent

model = NeuralNetwork().to(device)  # move model to GPU if present

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
print("Done!")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = []
y_true = []

for X, y in test_dataloader:
    pred = model(X)
    y_p = pred.argmax(1)
    y_pred.extend(y_p.numpy())
    y_true.extend(y.numpy())
    
# get labels for classes
classes = training_data.classes

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
cf_matrix = cf_matrix / np.sum(cf_matrix, axis=1)[:, None]  # compute percentages
disp = ConfusionMatrixDisplay(cf_matrix, display_labels=classes)

fig, ax = plt.subplots(figsize=(10,10))  # w/o this figure will overlap labels
disp.plot(ax=ax)

In [ ]:
model

Make sure you have graphviz and torchviz installed into the virtual environment.<br><br>
If you are using a GPU, I recommend moving model back to CPU for the torchviz section. The data and model need to be on same device, and that would be the simplest way.

In [ ]:
import torch
from torchviz import make_dot

# 1) Define inputs with gradients enabled
x = torch.tensor(3.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

# 2) Define a simple scalar function: f = x^2 - y
f = x**2 - y  # scalar tensor; grad_fn will be tracked

# (optional) Backprop just to populate .grad if you want to inspect later
f.backward()  # x.grad and y.grad now exist

In [ ]:
# 3) Build a Graphviz Digraph of the computation
dot = make_dot(f, params={"x": x, "y": y})

dot

In [ ]:
from torchviz import make_dot

X, y = training_data[0]
# model = model.to('cpu')
y_hat = model(torch.unsqueeze(X, dim=0))

# note grad_fn in output to see if you can
# find it in figure shown by make_dot()
y_hat   

In [ ]:
loss = loss_fn(y_hat, torch.tensor([y]))

In [ ]:
make_dot(loss, params=dict(model.named_parameters()))

make_dot() can give you more info if you set **show_attrs** and **show_saved** to **True**

In [ ]:
make_dot(loss, params=dict(model.named_parameters()), show_attrs=True, show_saved=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
learning_rate = 1e-3

model = CNN().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)
print("Done!")

In [ ]:
X, y = training_data[0]
y_hat = model(torch.unsqueeze(X, dim=0))    # torch is expecting batches

make_dot(y_hat, params=dict(model.named_parameters()))

## <center> References </center>

[1] pytorch: https://pytorch.org/

[2] How to Visualize PyTorch Neural Networks – 3 Examples in Python: https://appsilon.com/visualize-pytorch-neural-networks/

[3] Fashion MNIST with Pytorch (93% Accuracy) : https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy